In [1]:
import unidecode
from nltk.translate.bleu_score import sentence_bleu
import random as rd

aligned_othello_style1 = "data/shakespeare/data/align/plays/merged/othello_modern.snt.aligned"
aligned_othello_style2 = "data/shakespeare/data/align/plays/merged/othello_original.snt.aligned"

style1_tuples = unidecode.unidecode(open(aligned_style1).read())
style2_tuples = unidecode.unidecode(open(aligned_style2).read())

style1_tuples = style1_tuples.split("\n")
style2_tuples = style2_tuples.split("\n")

/Users/peterkong/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/peterkong/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/peterkong/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
style1_tuples[0]

"What's the matter, lieutenant?"

In [3]:
# in-class = original = '1'
# raw -> (raw text, label)
import numpy as np

style1_np = np.array(style1_tuples)
style1_labels = np.array(['1' for x in range(len(style1_tuples))])
X = np.vstack((style1_np, style1_labels))

style2_np = np.array(style2_tuples)
style2_labels = np.array(['0' for x in range(len(style2_tuples))])
X2 = np.vstack((style2_np, style2_labels))

X = np.hstack((X,X2))
raw = np.transpose(X)
raw.shape

idx_col = [str(x) for x in range(len(raw))]
idx_col = np.array(idx_col).reshape(-1,1)
print(idx_col.shape)
print(raw.shape)
raw = np.hstack((idx_col, raw))
raw.shape

(3710, 1)
(3710, 2)


(3710, 3)

In [4]:
raw[2000]

array(['2000', 'She is stirring, sir.', '0'], dtype='<U497')

In [5]:
# feature engineering
    # construct index column
idx_col = raw[:,0]
#idx_col.astype(float)

    
sent_len_feature = np.array([-1 for x in range(len(raw))])
raw_text_col = 1
LABEL_COL = -1

for row_idx,_ in enumerate(raw):
    sent_len_feature[row_idx] = len(raw[row_idx][raw_text_col])
#     if sent_len_feature[row_idx] < 0:
#         print("sent len less than zero!")

#sent_len_feature.astype(float)
X = np.vstack((idx_col,sent_len_feature, raw[:,LABEL_COL].astype(float)))
X = np.transpose(X)
X = X.astype('f', copy=True)
print(X[0:5])
print(X[3700:3705])

[[ 0. 30.  1.]
 [ 1. 53.  1.]
 [ 2. 15.  1.]
 [ 3. 29.  1.]
 [ 4. 30.  1.]]
[[3700.   40.    0.]
 [3701.   16.    0.]
 [3702.   21.    0.]
 [3703.   39.    0.]
 [3704.   67.    0.]]


In [11]:
X[0]

array([2856.,   30.,    0.], dtype=float32)

In [18]:
from sklearn.linear_model import LogisticRegression
import time
# training
# test breakout: 3500, 210
train_test_split_delim = 3500

# numpy note: addressing [:delim] is exclusive of delim, but
#                         [delim:] is inclusive of delim

np.random.permutation(X)
print(X[0:10])

X_train = X[0:train_test_split_delim,1]
X_train = X_train.reshape(-1, 1) #TODO: REMOVE
Y_train = X[0:train_test_split_delim,LABEL_COL]

X_test = X[train_test_split_delim:,1]
X_test = X_test.reshape(-1, 1)
Y_test = X[train_test_split_delim:,LABEL_COL]


logreg = LogisticRegression(random_state=123)
logreg.fit(X_train, Y_train)
X_train[:2]

[[2856.   30.    0.]
 [1862.   25.    0.]
 [  89.   66.    1.]
 [2752.   70.    0.]
 [1162.   20.    1.]
 [1693.   23.    1.]
 [3417.   97.    0.]
 [3203.   22.    0.]
 [1246.  130.    1.]
 [1160.    9.    1.]]


array([[30.],
       [25.]], dtype=float32)

In [28]:
# evaluation
preds = logreg.predict(X_test)
test_indices = X[train_test_split_delim:,0]
if len(preds) != len(test_indices):
    raise ValueError("unmatched arrays")
    
results = []
for (idx, pred) in zip(test_indices,preds):
    gold_label = int(raw[int(idx)][-1])
    pred_label = int(pred)
    #print("int'd idx: {} gold label: {} pred label: {}".format(int(idx), gold_label, pred_label))
    
    if gold_label == pred_label:
          results.append(1)
    else:
        results.append(0)
    
results[:5]


    

int'd idx: 1132 gold label: 1 pred label: 1
int'd idx: 2769 gold label: 0 pred label: 0
int'd idx: 435 gold label: 1 pred label: 0
int'd idx: 3625 gold label: 0 pred label: 1
int'd idx: 1248 gold label: 1 pred label: 1
int'd idx: 3467 gold label: 0 pred label: 1
int'd idx: 3369 gold label: 0 pred label: 0
int'd idx: 3278 gold label: 0 pred label: 1
int'd idx: 1651 gold label: 1 pred label: 0
int'd idx: 2345 gold label: 0 pred label: 1
int'd idx: 2661 gold label: 0 pred label: 0
int'd idx: 1374 gold label: 1 pred label: 0
int'd idx: 31 gold label: 1 pred label: 1
int'd idx: 3000 gold label: 0 pred label: 1
int'd idx: 3303 gold label: 0 pred label: 0
int'd idx: 2861 gold label: 0 pred label: 0
int'd idx: 707 gold label: 1 pred label: 1
int'd idx: 1303 gold label: 1 pred label: 1
int'd idx: 3503 gold label: 0 pred label: 1
int'd idx: 2248 gold label: 0 pred label: 1
int'd idx: 484 gold label: 1 pred label: 1
int'd idx: 2438 gold label: 0 pred label: 1
int'd idx: 1082 gold label: 1 pred la

In [31]:
test_len = len(preds)
accuracy = sum(preds)/float(test_len)
print("accuracy: {}".format(accuracy))

accuracy: 0.7428571428571429


In [106]:
#x = np.zeros(3, dtype=[('row_id','int32'),('text', 'S'), ('label', 'int32')])

In [16]:
np.set_printoptions(suppress=True)

py_arr = [[  3701.,  30.,   1.]
 ,[  3702.,  53.,   1.]
 ,[  3703.,  15.,   1.]]


arr = np.array(py_arr)
arr = arr.astype(float)
np.random.shuffle(arr)
time.sleep(1)
arr[:3]

array([[3702.,   53.,    1.],
       [3701.,   30.,    1.],
       [3703.,   15.,    1.]])

In [14]:
arr.dtype

dtype('float64')

In [25]:
import sys
sys.executable

'/Users/peterkong/miniconda3/bin/python'